In [1]:
# import os
# import sys
# sys.path.insert(0, os.path.join('..', 'app'))

In [2]:
cd ../app

/home/byl/Downloads/Program/Parking/app


In [3]:
from lib.data import ParkingData

DATA_DIR = 'data'

data_obj = ParkingData(data_dir=DATA_DIR)

23/01/10 18:58:10 WARN Utils: Your hostname, byl-i15-7000 resolves to a loopback address: 127.0.1.1; using 192.168.1.239 instead (on interface wlp3s0)
23/01/10 18:58:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/10 18:58:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/10 18:58:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/10 18:58:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
from config.data import SR_TABLE, BG_TABLE
from datetime import datetime

In [5]:
start_date = datetime(year=2020, month=4, day=1)
end_date = datetime(year=2020, month=4, day=30)
target_col = 'VehiclePresent'  # InViolation, VehiclePresent
order_col = 'DailyAverage'  # Total, DailyAverage
top_n = 10

TOPN_QUERY = """
SELECT 
       BayId
     , DATEDIFF(MAX(DepartureTime), MIN(ArrivalTime))+1 AS OperatingDays
     , SUM({target_col}) AS Total
     , SUM({target_col}) / (DATEDIFF(MAX(DepartureTime), MIN(ArrivalTime))+1) AS DailyAverage
FROM {table}
WHERE {target_col} = 1
  AND CAST(DepartureTime AS DATE) >= DATE('{start_date}')
  AND CAST(ArrivalTime AS DATE) <= DATE('{end_date}')
GROUP BY BayId
ORDER BY {order_col} DESC
LIMIT {top_n}
"""

data_obj.query(TOPN_QUERY.format(
    table=SR_TABLE,
    start_date=start_date,
    end_date=end_date,
    target_col=target_col,
    order_col=order_col,
    top_n=top_n,
))

,BayId,OperatingDays,Total,DailyAverage
0,4324,25,3404,136.160000
1,2287,31,1271,41.000000
2,4046,30,1141,38.033333
3,6003,1,36,36.000000
4,2152,31,995,32.096774
5,5983,10,303,30.300000
6,6004,32,904,28.250000
7,2674,30,834,27.800000
8,2144,31,846,27.290323
9,1762,30,727,24.233333


In [6]:
start_date = datetime(year=2020, month=4, day=1)
end_date = datetime(year=2020, month=4, day=30)
target_col = 'VehiclePresent'  # InViolation, VehiclePresent

HEATMAP_QUERY = """
SELECT SR.BayId
     , BG.latlong[0] AS latitude
     , BG.latlong[1] AS longitude
     , COUNT(*) AS total
FROM {sr_table} SR
LEFT JOIN {bg_table} BG
    ON SR.BayId = BG.bay_id
WHERE ArrivalTime <= '{end_date}'
  AND SR.DepartureTime >= '{start_date}'
  AND SR.{target_col} = 1
GROUP BY SR.BayId
       , BG.latlong
"""

data_obj.query(HEATMAP_QUERY.format(
    sr_table=SR_TABLE,
    bg_table=BG_TABLE,
    start_date=start_date,
    end_date=end_date,
    target_col=target_col,
))

,BayId,latitude,longitude,total
0,5537,-37.799749,144.944601,170
1,5902,-37.803749,144.909310,47
2,5510,-37.841137,144.978510,64
3,5144,-37.810245,144.945769,27
4,5092,-37.841144,144.978480,12
...,...,...,...,...
1256,8797,-37.813393,144.954666,1
1257,3802,-37.791829,144.921716,105
1258,2530,-37.796193,144.974138,120
1259,4390,-37.811286,144.945704,65
